## Getting the Access Code for Slack

The code below illustrates how we connect to the Slack API to request an authorization token for our app. Remember that we have to register our app with Slack first, and get the `client_id`.

#### Creating a Slack App

* Go to https://api.slack.com/apps and create your app. You will need the "Client ID" and the "Client Secret" that is created for you.
* Select the **"OAuth & Permissions"** tab from the left-hand side and add a "Redirect URL" for your app. The redirect URL ensures (for security) that the app can only talk to your own web server. Add `http://<yourIP>:5000/slack` as your redirect URL.
* Select the "Bot Users" tab from the left-hand side and add a bot username for your app.


#### Get the Web Server up and running

* See the Python Script `webserver.py` that is in this folder. 
* **IMPORTANT**: Modify the CLIENT_ID, CLIENT_SECRET, and REDIRECT variables in `slack_app.json` to match those of your own Slack app.
* You now need to start the server, so that it can receive the authentication code for the user. 

In [ ]:
# the command below will run the webserver from the notebook
# Notice that the server is configured to stop running 
# after receiving the first authorization grant and storing the access token
%run webserver.py

In [ ]:
# Alternatively, you can launch the webserver from the Terminal issuing the command
#
# python3 webserver.py


In [ ]:
# Alternatively, if you want to run the server in the background, type:
#
# nohup ./webserver.py &
#
# which will put the server to run in the background
#
# If you need to stop the background server, you can isssue the following
# command from the terminal, which will stop any process that contains 
# `WebServer` as part of its name
#
#  kill $(pgrep WebServer)

Now, we are ready to execute the authentication flow, which is illustrated in the picture below.

<img width = "800" src="https://a.slack-edge.com/bfaba/img/api/slack_oauth_flow_diagram@2x.png">

#### Launch the user authentication process (Steps 1-4 in the picture above)

* Now to go `http://<yourIP>:5000/install` and click the "Install Slack Bot" URL. You will be asked to login to your Slack account, and grant permissions to the bot to use your account credentials/permissions. This is the Step 1 in the picture above.

* Once you grant permissions, then then the Slack server will call the `http://<yourIP>:5000/slack` URL and send the authentication code to the redirect URL. This is Step 2 in the picture above.

* At that point, out web server, will call back Slack Service API saying "hey, I got the code, here is the verification (my `client_secret`) that I am indeed the correct app. Can you send me the access token for the user?" (See the `WebServer.py` code, under the "/slack" route.) This is Step 3 in the picture above.

* The Slack service API will send us back the "access token" for the authenticated user. We will save this in the file `slack_secret.json`. This is Step 4 in the picture above.

* The webserver will stop running after a successful authorization, and the cell above will stop showing a `[*]`

#### Done with the authentication, the authentication data is now stored locally.

After we authorize the app, our web server has received from Slack the authentication token, and it was stored in the `slack_secret.json` file. Now, we are ready to proceed with Steps 5 and 6, which are the regular API calls to the Slack API.

In [ ]:
import json

# Read the access token from the file
OAUTH_FILE = 'slack_secret.json'
f = open(OAUTH_FILE, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)

auth_info

In [1]:
# We will just use the library to test that the code that we get back
# allows us to connect to the Slack API
!sudo -H python3 -m pip install slackclient

In [ ]:
from slackclient import SlackClient
sc = SlackClient(auth_info["access_token"])

In [ ]:
response = sc.api_call("users.info", user=auth_info["user_id"])
user = response['user']

In [ ]:
print("The username of the authenticated user is", user.get('name'))
print("The email of the authenticated user is", user.get('profile').get('email'))
print("The email of the authenticated user is", user.get('profile').get('real_name'))